In [12]:
# dataset.py
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset


class ISICDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

        # ISIC 2019 class columns
        self.class_names = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC"]

        # Convert one-hot → class index
        self.df["label"] = self.df[self.class_names].values.argmax(axis=1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]["image"]
        label = int(self.df.iloc[idx]["label"])

        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# model.py
import torch.nn as nn
from torchvision import models


class DenseNetClassifier(nn.Module):
    def __init__(
        self,
        num_classes=7,
        freeze_backbone=True,
        dropout=0.5,
    ):
        super().__init__()

        self.model = models.densenet121(weights="IMAGENET1K_V1")

        if freeze_backbone:
            for param in self.model.features.parameters():
                param.requires_grad = False

        in_features = self.model.classifier.in_features

        # Custom classifier head
        self.model.classifier = nn.Sequential(
            nn.BatchNorm1d(in_features),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        return self.model(x)

In [35]:
model = DenseNetClassifier(num_classes=7).to("cpu")

print(model)

DenseNetClassifier(
  (model): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, mom

In [15]:
# trainer.py
import torch
from tqdm import tqdm


class Trainer:
    def __init__(self, model, device, optimizer, criterion):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

    def train_one_epoch(self, loader):
        self.model.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in tqdm(loader, desc="Training"):
            images, labels = images.to(self.device), labels.to(self.device)

            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)

            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        return total_loss / len(loader), correct / total

    def validate(self, loader):
        self.model.eval()
        total_loss, correct, total = 0, 0, 0

        with torch.no_grad():
            for images, labels in tqdm(loader, desc="Validation"):
                images, labels = images.to(self.device), labels.to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, labels)

                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).sum().item()
                total += labels.size(0)

        return total_loss / len(loader), correct / total

In [21]:
# train.py
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn, optim


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )

    train_dataset = ISICDataset(
        csv_file="../data/slice/ISIC_SUBSET/train.csv",
        image_dir="../data/slice/ISIC_SUBSET/images",
        transform=transform,
    )

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = DenseNetClassifier(num_classes=7).to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.AdamW(
        [
            {"params": model.model.features.parameters(), "lr": 1e-4},
            {"params": model.model.classifier.parameters(), "lr": 1e-3},
        ]
    )
    trainer = Trainer(model, device, optimizer, criterion)

    epochs = 5
    best_loss = float("inf")
    for epoch in range(epochs):
        train_loss, train_acc = trainer.train_one_epoch(train_loader)

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"Train Loss: {train_loss:.4f} "
            f"Train Acc: {train_acc:.4f}"
        )

        if train_loss < best_loss:
            best_loss = train_loss
            torch.save(
                {
                    "model_name": "densenet121",
                    "num_classes": 7,
                    "freeze_backbone": True,
                    "state_dict": model.state_dict(),
                },
                # f"densenet_isic_{epoch:02d}_{train_acc:.4f}.pth",
                "densenet_isic.pth",
            )


main()

Training: 100%|██████████| 32/32 [02:50<00:00,  5.34s/it]


Epoch [1/5] Train Loss: 1.1603 Train Acc: 0.5775


Training: 100%|██████████| 32/32 [02:39<00:00,  4.99s/it]


Epoch [2/5] Train Loss: 0.8954 Train Acc: 0.6625


Training: 100%|██████████| 32/32 [05:31<00:00, 10.37s/it]


Epoch [3/5] Train Loss: 0.8347 Train Acc: 0.6970


Training: 100%|██████████| 32/32 [08:53<00:00, 16.68s/it]


Epoch [4/5] Train Loss: 0.7985 Train Acc: 0.7000


Training: 100%|██████████| 32/32 [08:55<00:00, 16.74s/it]

Epoch [5/5] Train Loss: 0.7532 Train Acc: 0.7270


# Validation


In [ ]:
# validate_isic_onehot.py
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
from pathlib import Path

# =====================
# CONFIG
# =====================
MODEL_PATH = "densenet_isic.pth"
IMAGE_DIR = "../data/slice/ISIC_SUBSET/images"
VAL_CSV = "../data/slice/ISIC_SUBSET/val.csv"
IMG_SIZE = 224

# Only include the classes used for training
CLASS_NAMES = ["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC"]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =====================
# IMAGE TRANSFORM
# =====================
transform = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# =====================
# LOAD MODEL
# =====================
def load_model(path):
    checkpoint = torch.load(path, map_location=DEVICE)
    model = DenseNetClassifier(num_classes=len(CLASS_NAMES), freeze_backbone=False).to(
        DEVICE
    )
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    return model


# =====================
# PREDICT SINGLE IMAGE
# =====================
def predict(model, img_path):
    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        outputs = model(img)
        pred_idx = outputs.argmax(1).item()
        confidence = torch.softmax(outputs, dim=1)[0, pred_idx].item()
    return pred_idx, CLASS_NAMES[pred_idx], confidence


# =====================
# MAIN VALIDATION
# =====================
def main():
    model = load_model(MODEL_PATH)
    df = pd.read_csv(VAL_CSV)

    total = 0
    correct = 0
    results = []

    for _, row in df.iterrows():
        img_name = row["image"].strip() + ".jpg"
        img_path = Path(IMAGE_DIR) / img_name
        if not img_path.exists():
            print(f"WARNING: {img_path} not found, skipping.")
            continue

        pred_idx, pred_class, conf = predict(model, img_path)

        # --- ground truth from one-hot columns ---
        gt_idx = row[1 : 1 + len(CLASS_NAMES)].values.argmax()
        gt_class = CLASS_NAMES[gt_idx]

        total += 1
        correct += int(pred_idx == gt_idx)

        results.append(
            {
                "image": img_name,
                "gt_label": gt_class,
                "prediction": pred_class,
                "confidence": round(conf, 3),
            }
        )

        print(f"{img_name}: GT={gt_class} | PRED={pred_class} ({conf:.2f})")

    # --- final accuracy ---
    val_acc = correct / total if total > 0 else 0
    print(f"\nValidation Accuracy: {val_acc:.4f}")

    # --- save predictions ---
    pd.DataFrame(results).to_csv("predictions_val.csv", index=False)
    print("Predictions saved to predictions_val.csv")


if __name__ == "__main__":
    main()

ISIC_0054697.jpg: GT=BCC | PRED=BKL (0.37)
ISIC_0000360.jpg: GT=NV | PRED=NV (0.74)
ISIC_0059394.jpg: GT=BCC | PRED=BCC (0.78)
ISIC_0026375.jpg: GT=NV | PRED=NV (0.99)
ISIC_0059292.jpg: GT=NV | PRED=NV (0.63)
ISIC_0031400.jpg: GT=BCC | PRED=BCC (1.00)
ISIC_0028987.jpg: GT=NV | PRED=NV (0.81)
ISIC_0055169.jpg: GT=MEL | PRED=MEL (0.97)
ISIC_0068495.jpg: GT=MEL | PRED=MEL (0.80)
ISIC_0031602.jpg: GT=NV | PRED=NV (0.74)
ISIC_0066810.jpg: GT=BKL | PRED=BKL (0.42)
ISIC_0063352.jpg: GT=BCC | PRED=BCC (0.65)
ISIC_0025657.jpg: GT=NV | PRED=NV (0.82)
ISIC_0064072.jpg: GT=MEL | PRED=MEL (0.79)
ISIC_0073021.jpg: GT=BKL | PRED=BCC (0.58)
ISIC_0060204.jpg: GT=BKL | PRED=BCC (0.34)
ISIC_0032034.jpg: GT=NV | PRED=NV (0.98)
ISIC_0072668.jpg: GT=NV | PRED=NV (0.90)
ISIC_0066796.jpg: GT=NV | PRED=NV (0.74)
ISIC_0031807.jpg: GT=NV | PRED=NV (0.99)
ISIC_0069383.jpg: GT=NV | PRED=NV (0.65)
ISIC_0064521.jpg: GT=MEL | PRED=MEL (0.82)
ISIC_0053685.jpg: GT=MEL | PRED=BCC (0.35)
ISIC_0060406.jpg: GT=MEL | PRED=M